<a href="https://colab.research.google.com/github/benjaminbrown038/Amazon/blob/main/notebooks/amazon/amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon

- Hugging Face
- PyTorch
- Tensorflow

## Sagemaker

#### Imports

In [1]:
%%capture
!pip3 install sagemaker
import logging
import time

import boto3

import sagemaker
from sagemaker.xgboost import XGBoost
from sagemaker.experiments.run import Run
from sagemaker.analytics import ExperimentAnalytics

In [2]:
role = sagemaker.get_execution_role()
role

ValueError: ignored

In [ ]:
region = boto3.Session().region_name
region

In [ ]:
sagemaker_session = sagemaker.Session()
sagemaker_session

In [ ]:
bucket_name = sagemaker_session.default_bucket()
bucket_name

In [ ]:
prefix = "end-to-end-ml"

## Hugging Face

#### Imports

In [4]:
%%capture
!pip3 install sagemaker --upgrade
!pip3 install datasets

import sagemaker
from sagemaker.huggingface import HuggingFace, TrainingCompilerConfig
import sagemaker
from sagemaker.huggingface import HuggingFace
from sagemaker.s3 import S3Downloader
import transformers
import datasets

import boto3


import argparse
import os

In [ ]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='role-name-of-your-iam-role-with-right-permissions')['Role']['Arn']
sess = sagemaker.Session()

In [ ]:
if __name__ == "__main__":

    parser = argparse.ArgumentParser()

    parser.add_argument("--epochs", type=int, default=3)
    parser.add_argument("--per_device_train_batch_size", type=int, default=32)
    parser.add_argument("--model_name_or_path", type=str)

    parser.add_argument("--model-dir", type=str, default=os.environ["SM_MODEL_DIR"])
    parser.add_argument("--training_dir", type=str, default=os.environ["SM_CHANNEL_TRAIN"])
    parser.add_argument("--test_dir", type=str, default=os.environ["SM_CHANNEL_TEST"])

In [5]:
hyperparameters={'epochs': 1,
                 'per_device_train_batch_size': 32,
                 'model_name_or_path': 'distilbert-base-uncased'}
hyperparameters

{'epochs': 1,
 'per_device_train_batch_size': 32,
 'model_name_or_path': 'distilbert-base-uncased'}

In [6]:
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='./scripts',
        instance_type='ml.p3.2xlarge',
        instance_count=1,
        role=role,
        transformers_version='4.26',
        pytorch_version='1.13',
        py_version='py39',
        hyperparameters = hyperparameters)
huggingface_estimator

NameError: ignored

In [ ]:
huggingface_estimator.fit(
  {'train': 's3://sagemaker-us-east-1-558105141721/samples/datasets/imdb/train',
   'test': 's3://sagemaker-us-east-1-558105141721/samples/datasets/imdb/test'})

In [ ]:
/opt/conda/bin/python train.py --epochs 1 --model_name_or_path distilbert-base-uncased --per_device_train_batch_size 32

In [ ]:
S3Downloader.download(
    s3_uri=huggingface_estimator.model_data,
    local_path='.',
    sagemaker_session=sess)

In [7]:
distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}
distribution

{'smdistributed': {'dataparallel': {'enabled': True}}}

In [ ]:
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='./scripts',
        instance_type='ml.p3dn.24xlarge',
        instance_count=2,
        role=role,
        transformers_version='4.26.0',
        pytorch_version='1.13.1',
        py_version='py39',
        hyperparameters = hyperparameters,
        distribution = distribution)
huggingface_estimator

In [8]:
mpi_options = {
    "enabled" : True,
    "processes_per_host" : 8}
mpi_options

{'enabled': True, 'processes_per_host': 8}

In [9]:
smp_options = {
        "enabled":True,
        "parameters": {
        "microbatches": 4,
        "placement_strategy": "spread",
        "pipeline": "interleaved",
        "optimize": "speed",
        "partitions": 4,
        "ddp": True}}
smp_options

{'enabled': True,
 'parameters': {'microbatches': 4,
  'placement_strategy': 'spread',
  'pipeline': 'interleaved',
  'optimize': 'speed',
  'partitions': 4,
  'ddp': True}}

In [10]:
distribution={
    "smdistributed": {"modelparallel": smp_options},
    "mpi": mpi_options}
distribution

{'smdistributed': {'modelparallel': {'enabled': True,
   'parameters': {'microbatches': 4,
    'placement_strategy': 'spread',
    'pipeline': 'interleaved',
    'optimize': 'speed',
    'partitions': 4,
    'ddp': True}}},
 'mpi': {'enabled': True, 'processes_per_host': 8}}

In [ ]:
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='./scripts',
        instance_type='ml.p3dn.24xlarge',
        instance_count=2,
        role=role,
        transformers_version='4.26.0',
        pytorch_version='1.13.1',
        py_version='py39',
        hyperparameters = hyperparameters,
        distribution = distribution)
huggingface_estimator

In [11]:
hyperparameters={'epochs': 1,
                 'train_batch_size': 32,
                 'model_name':'distilbert-base-uncased',
                 'output_dir':'/opt/ml/checkpoints'}
hyperparameters

{'epochs': 1,
 'train_batch_size': 32,
 'model_name': 'distilbert-base-uncased',
 'output_dir': '/opt/ml/checkpoints'}

In [ ]:
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='./scripts',
        instance_type='ml.p3.2xlarge',
        instance_count=1,
	      checkpoint_s3_uri=f's3://{sess.default_bucket()}/checkpoints'
        use_spot_instances=True,
        max_wait=3600,
        max_run=1000,
        role=role,
        transformers_version='4.26',
        pytorch_version='1.13',
        py_version='py39',
        hyperparameters = hyperparameters)
huggingface_estimator

In [12]:
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.4.2'}
git_config

{'repo': 'https://github.com/huggingface/transformers.git', 'branch': 'v4.4.2'}

In [ ]:
huggingface_estimator = HuggingFace(
        entry_point='run_glue.py',
        source_dir='./examples/pytorch/text-classification',
        git_config=git_config,
        instance_type='ml.p3.2xlarge',
        instance_count=1,
        role=role,
        transformers_version='4.26',
        pytorch_version='1.13',
        py_version='py39',
        hyperparameters=hyperparameters)
huggingface_estimator

In [13]:
metric_definitions = [
    {"Name": "train_runtime", "Regex": "train_runtime.*=\D*(.*?)$"},
    {"Name": "eval_accuracy", "Regex": "eval_accuracy.*=\D*(.*?)$"},
    {"Name": "eval_loss", "Regex": "eval_loss.*=\D*(.*?)$"}]
metric_definitions

[{'Name': 'train_runtime', 'Regex': 'train_runtime.*=\\D*(.*?)$'},
 {'Name': 'eval_accuracy', 'Regex': 'eval_accuracy.*=\\D*(.*?)$'},
 {'Name': 'eval_loss', 'Regex': 'eval_loss.*=\\D*(.*?)$'}]

In [ ]:
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='./scripts',
        instance_type='ml.p3.2xlarge',
        instance_count=1,
        role=role,
        transformers_version='4.26',
        pytorch_version='1.13',
        py_version='py39',
        metric_definitions=metric_definitions,
        hyperparameters = hyperparameters)
huggingface_estimator

In [ ]:
huggingface_estimator=HuggingFace(
    compiler_config=TrainingCompilerConfig())
huggingface_estimator

# PyTorch
### Use PyTorch with the SageMaker Python SDK

- Train

- Deploy

In [17]:
%%capture

import argparse
import os

import torch
import torch.distributed as dist
from sagemaker.pytorch import PyTorch

from six import BytesIO


from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow
from sagemaker.tensorflow import TensorFlowModel

In [18]:
if __name__ =='__main__':

    parser = argparse.ArgumentParser()


    parser.add_argument('--epochs', type=int, default=50)
    parser.add_argument('--batch-size', type=int, default=64)
    parser.add_argument('--learning-rate', type=float, default=0.05)
    parser.add_argument('--use-cuda', type=bool, default=False)

    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    parser.add_argument('--test', type=str, default=os.environ['SM_CHANNEL_TEST'])

    args, _ = parser.parse_known_args()

KeyError: ignored

In [ ]:
if __name__=='__main__':

    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    args, _ = parser.parse_known_args()


    with open(os.path.join(args.model_dir, 'model.pth'), 'wb') as f:
        torch.save(model.state_dict(), f)

In [19]:
model_dir = os.path.join(model_dir, "model.pt")
model_dir

NameError: ignored

In [ ]:
torch.jit.save(model, model_dir)

In [ ]:
pytorch_estimator = PyTorch('pytorch-train.py',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            framework_version='1.8.0',
                            py_version='py3',
                            hyperparameters = {'epochs': 20, 'batch-size': 64, 'learning-rate': 0.1})
pytorch_estimator

In [ ]:
pytorch_estimator.fit({'train': 's3://my-data-bucket/path/to/my/training/data',
                       'test': 's3://my-data-bucket/path/to/my/test/data'})

In [20]:
{'train':'s3://my-bucket/my-training-data',
 'eval':'s3://my-bucket/my-evaluation-data'}

{'train': 's3://my-bucket/my-training-data',
 'eval': 's3://my-bucket/my-evaluation-data'}

In [ ]:
if args.distributed:
    world_size = len(args.hosts)
    os.environ['WORLD_SIZE'] = str(world_size)
    host_rank = args.hosts.index(args.current_host)
    dist.init_process_group(backend=args.backend, rank=host_rank)

In [ ]:
pt_estimator = PyTorch(
    entry_point="train_ptddp.py",
    role="SageMakerRole",
    framework_version="1.12.0",
    py_version="py38",
    instance_count=2,
    instance_type="ml.p4d.24xlarge",
    distribution={
        "pytorchddp": {
            "enabled": True}})
pt_estimator

In [ ]:
pt_estimator = PyTorch(
    entry_point="train_ptddp.py",
    role="SageMakerRole",
    framework_version="1.13.1",
    py_version="py38",
    instance_count=2,
    instance_type="ml.p4d.24xlarge",
    distribution={
        "torch_distributed": {
        "enabled": True}})
pt_estimator

In [ ]:
dist.init_process_group('xla')

In [ ]:
pt_estimator = PyTorch(
    entry_point="train_torch_distributed.py",
    role="SageMakerRole",
    framework_version="1.11.0",
    py_version="py38",
    instance_count=1,
    instance_type="ml.trn1.2xlarge",
    distribution={
        "torch_distributed": {
            "enabled": True}})
pt_estimator

In [ ]:
pt_estimator.fit("s3://bucket/path/to/training/data")

In [ ]:
pt_estimator = PyTorch(
    entry_point="train_torch_distributed.py",
    role="SageMakerRole",
    framework_version="1.11.0",
    py_version="py38",
    instance_count=2,
    instance_type="ml.trn1.32xlarge",
    distribution={
        "torch_distributed": {
            "enabled": True}})
pt_estimator

In [ ]:
pt_estimator.fit("s3://bucket/path/to/training/data")

In [ ]:
pytorch_estimator = PyTorch(entry_point='train_and_deploy.py',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            framework_version='1.8.0',
                            py_version='py3')
pytorch_estimator

In [ ]:
pytorch_estimator.fit('s3://my_bucket/my_training_data/')

In [21]:
predictor = pytorch_estimator.deploy(instance_type='ml.m4.xlarge',initial_instance_count=1)
predictor

NameError: ignored

In [22]:
response = predictor.predict(data)
response

NameError: ignored

In [ ]:
predictor = pytorch_estimator.deploy(instance_type='ml.m4.xlarge',
                                     initial_instance_count=1,
                                     accelerator_type='ml.eia2.medium')
predictor

In [23]:
def model_fn(model_dir, context)

SyntaxError: ignored

In [24]:
def model_fn(model_dir, context):
    model = Your_Model()
    with open(os.path.join(model_dir, 'model.pth'), 'rb') as f:
        model.load_state_dict(torch.load(f))
    return model

In [25]:
def model_fn(model_dir):
    model = torch.jit.load('model.pth', map_location=torch.device('cpu'))
    if torch.__version__ == '1.5.1':
        import torcheia
        model = model.eval()
        model = torcheia.jit.attach_eia(model, 0)
    return model

In [26]:
input_object = input_fn(request_body, request_content_type, context)
input_object

NameError: ignored

In [ ]:
prediction = predict_fn(input_object, model, context)
prediction

In [ ]:
output = output_fn(prediction, response_content_type, context)
output

In [ ]:
def input_fn(request_body, request_content_type):
    if request_content_type == 'application/python-pickle':
        return torch.load(BytesIO(request_body))
    else:
        pass

In [ ]:
def predict_fn(input_object, model, context)

In [27]:
def predict_fn(input_data, model):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    with torch.no_grad():
        return model(input_data.to(device))

In [28]:
def predict_fn(input_data, model, context):
    device = torch.device("cuda:" + str(context.system_properties.get("gpu_id")) if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    with torch.no_grad():
        return model(input_data.to(device))

In [29]:
def predict_fn(input_data, model):
    device = torch.device("cpu")
    model = model.to(device)
    input_data = data.to(device)
    model.eval()
    with torch.jit.optimized_execution(True, {"target_device": "eia:0"}):
        output = model(input_data)

In [30]:
def predict_fn(input_data, model):
    device = torch.device("cpu")
    input_data = data.to(device)
    torch._C._jit_set_profiling_executor(False)
    with torch.jit.optimized_execution(True):
        output = model.forward(input_data)

In [32]:
def output_fn(prediction, content_type, context):

SyntaxError: ignored

In [ ]:
role = get_execution_role()
role

In [ ]:
pytorch_model = PyTorchModel(model_data='s3://my-bucket/my-path/model.tar.gz', role=role,entry_point='inference.py')
pytorch_model

In [ ]:
predictor = pytorch_model.deploy(instance_type='ml.c4.xlarge', initial_instance_count=1)
predictor

In [ ]:
my_training_job_name = 'MyAwesomePyTorchTrainingJob'
my_training_job_name

In [ ]:
pytorch_estimator = PyTorch.attach(my_training_job_name)
pytorch_estimator

## Tensorflow

#### Deploying to TensorFlow Serving Endpoints

In [ ]:
estimator = TensorFlow(
    entry_point="tf-train.py",
    instance_count=1,
    instance_type="ml.c4.xlarge",
    framework_version="2.2",
    py_version="py37")
estimator

In [ ]:
estimator.fit(inputs)

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.c5.xlarge")
predictor

In [ ]:
model = TensorFlowModel(model_data='s3://mybucket/model.tar.gz', role='MySageMakerRole')
model

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')
predictor

In [ ]:
model = TensorFlowModel(model_data='s3://mybucket/model.tar.gz', role='MySageMakerRole')
model

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge', accelerator_type='ml.eia1.medium')
predictor

In [ ]:
input = {'instances': [1.0, 2.0, 5.0]}
input

In [ ]:
result = predictor.predict(input)
result

In [ ]:
{'predictions': [3.5, 4.0, 5.5]}

In [ ]:
input = {'signature_name': 'tensorflow/serving/regress',
         'examples': [{'x': 1.0}, {'x': 2.0}]}
input

In [ ]:
result = predictor.regress(input)
result

In [ ]:
{'results': [3.5, 4.0]}

In [ ]:
input = {'instances': [
    [1.0, 2.0, 5.0],
    [1.0, 2.0, 5.0],
    [1.0, 2.0, 5.0]]}
input

In [ ]:
result = predictor.predict(input)
result

In [ ]:
{'predictions': [
  [3.5, 4.0, 5.5],
  [3.5, 4.0, 5.5],
  [3.5, 4.0, 5.5]]}

In [ ]:
input = [[1.0, 2.0, 5.0],
        [1.0, 2.0, 5.0]]
input

In [ ]:
result = predictor.predict(input)
result

In [ ]:
{'predictions': [
  [3.5, 4.0, 5.5],
  [3.5, 4.0, 5.5]]}

In [ ]:
input = {'x': [1.0, 2.0, 5.0]}
input

In [ ]:
{'predictions': [[3.5, 4.0, 5.5]]}

In [ ]:
predictor = Predictor('endpoint-name', serializer=None, content_type='application/jsonlines')
predictor

In [ ]:
input = '''{'x': [1.0, 2.0, 5.0]}
{'x': [1.0, 2.0, 5.0]}
{'x': [1.0, 2.0, 5.0]}'''
input

In [ ]:
result = predictor.predict(input)
result

In [ ]:
{'predictions': [
  [3.5, 4.0, 5.5],
  [3.5, 4.0, 5.5],
  [3.5, 4.0, 5.5]]}

In [ ]:
predictor = Predictor('endpoint-name', serializer=sagemaker.serializers.CSVSerializer())
predictor

In [ ]:
input = '1.0,2.0,5.0\n1.0,2.0,5.0\n1.0,2.0,5.0'
input

In [ ]:
result = predictor.predict(input)
result

In [ ]:
{'predictions': [
  [3.5, 4.0, 5.5],
  [3.5, 4.0, 5.5],
  [3.5, 4.0, 5.5]]}

In [ ]:
model = Model(entry_point='inference.py',
              model_data='s3://mybucket/model.tar.gz',
              role='MySageMakerRole')
model

In [ ]:
def input_handler(data, context):

    if context.request_content_type == 'application/json':
        d = data.read().decode('utf-8')
        return d if len(d) else ''

    if context.request_content_type == 'text/csv':
        return json.dumps({
            'instances': [float(x) for x in data.read().decode('utf-8').split(',')]})

    raise ValueError('{{"error": "unsupported content type {}"}}'.format(
        context.request_content_type or "unknown"))


def output_handler(data, context):
    if data.status_code != 200:
        raise ValueError(data.content.decode('utf-8'))
    response_content_type = context.accept_header
    prediction = data.content
    return prediction, response_content_type

In [ ]:
def handler(data, context):
    processed_input = _process_input(data, context)
    response = requests.post(context.rest_uri, data=processed_input)
    return _process_output(response, context)

def _process_input(data, context):
    if context.request_content_type == 'application/json':

        d = data.read().decode('utf-8')
        return d if len(d) else ''

    if context.request_content_type == 'text/csv':

        return json.dumps({
            'instances': [float(x) for x in data.read().decode('utf-8').split(',')]})

    raise ValueError('{{"error": "unsupported content type {}"}}'.format(
        context.request_content_type or "unknown"))

def _process_output(data, context):
    if data.status_code != 200:
        raise ValueError(data.content.decode('utf-8'))

    response_content_type = context.accept_header
    prediction = data.content
    return prediction, response_content_type

In [ ]:
model = Model(entry_point='inference.py',
              source_dir='source/directory',
              model_data='s3://mybucket/model.tar.gz',
              role='MySageMakerRole')
model

In [ ]:
model = Model(entry_point='inference.py',
              dependencies=['/path/to/folder/named/lib'],
              model_data='s3://mybucket/model.tar.gz',
              role='MySageMakerRole')
model

In [ ]:
aws s3 cp s3://mybucket/models/model1/model.tar.gz model1.tar.gz

In [ ]:
aws s3 cp s3://mybucket/models/model2/model.tar.gz model2.tar.gz

In [ ]:
mkdir -p multi/model1

In [ ]:
mkdir -p multi/model2

In [ ]:
tar xvf model1.tar.gz -C ./multi/model1

In [ ]:
tar xvf model2.tar.gz -C ./multi/model2

In [ ]:
mv multi/model1/export/Servo/* multi/model1/

In [ ]:
mv multi/model2/export/Servo/* multi/model2/

In [ ]:
rm -fr multi/model1/export

In [ ]:
rm -fr multi/model2/export

In [ ]:
tar -C "$PWD/multi/" -czvf multi.tar.gz multi/

In [ ]:
aws s3 cp multi.tar.gz s3://mybucket/models/multi.tar.gz

In [ ]:
role = 'SageMakerRole'
role

In [ ]:
model_data = 's3://mybucket/models/multi.tar.gz'
model_data

In [ ]:
env = {'SAGEMAKER_TFS_DEFAULT_MODEL_NAME': 'model1'}
env

In [ ]:
model = Model(model_data=model_data, role=role, framework_version='1.11', env=env)
model

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')
predictor

In [ ]:
endpoint = predictor.endpoint_name
endpoint

In [ ]:
model2_predictor = Predictor(endpoint, model_name='model2')
model2_predictor

In [ ]:
result = model2_predictor.predict()
result

In [ ]:

aws sagemaker-runtime invoke-endpoint \
    --endpoint-name my-endpoint \
    --content-type 'application/json' \
    --body '{"instances": [1.0, 2.0, 5.0]}' \
    >(cat) 1>/dev/null

# Predict request for specific model name
aws sagemaker-runtime invoke-endpoint \
    --endpoint-name my-endpoint \
    --content-type 'application/json' \
    --body '{"instances": [1.0, 2.0, 5.0]}' \
    --custom-attributes 'tfs-model-name=other_model' \
    >(cat) 1>/dev/null

# TensorFlow Serving REST API - regress request
aws sagemaker-runtime invoke-endpoint \
    --endpoint-name my-endpoint \
    --content-type 'application/json' \
    --body '{"signature_name": "tensorflow/serving/regress","examples": [{"x": 1.0}]}' \
    --custom-attributes 'tfs-method=regress' \
    >(cat) 1>/dev/null

# Simple json request (2 instances)
aws sagemaker-runtime invoke-endpoint \
    --endpoint-name my-endpoint \
    --content-type 'application/json' \
    --body '[[1.0, 2.0, 5.0],[2.0, 3.0, 4.0]]' \
    >(cat) 1>/dev/null

# CSV request (2 rows)
aws sagemaker-runtime invoke-endpoint \
    --endpoint-name my-endpoint \
    --content-type 'text/csv' \
    --body "1.0,2.0,5.0"$'\n'"2.0,3.0,4.0" \
    >(cat) 1>/dev/null

# Line delimited JSON from an input file
aws sagemaker-runtime invoke-endpoint \
    --endpoint-name my-endpoint \
    --content-type 'application/jsons' \
    --body "$(cat input.jsons)" \
    results.json

# Tensorflow
